In [1]:
import networkx as nx
import polars as pl

In [2]:
ds = pl.read_csv("dataset_original.csv")
print(len(ds))
ds.head(10)

10000000


_id,given_name,surname,suburb,postcode
str,str,str,str,str
"""0#0""","""annie""","""johnson""","""warrenron""","""2758g"""
"""0#1""","""erib""","""pugh""","""asheboro""","""27202"""
"""0#2""","""lauren""","""toledo""","""chapeo hill""","""2751q"""
"""0#3""","""keorgetta""","""atkin5on""","""goldsboro""","""27530"""
"""0#4""","""lvnn""","""kitchenz""","""greensboro""","""27408"""
"""0#5""","""jacqveline""","""lacewell""","""riegelwood""","""284s6"""
"""0#6""","""mayr""","""mmartin""","""charlotte""","""28205"""
"""0#7""","""geracld""","""clofntz""","""midland""","""28107"""
"""0#8""","""marie""","""roact""","""reisdville""","""27320"""


In [3]:
value_counter = dict()
for a in ds.columns[1:]:
    value_counter[a] = dict(zip(*ds[a].value_counts().to_dict(as_series=False).values()))

In [4]:
records = {row["_id"]: row for row in ds.to_dicts()}
print(len(records))
records["0#0"]

10000000


{'_id': '0#0',
 'given_name': 'annie',
 'surname': 'johnson',
 'suburb': 'warrenron',
 'postcode': '2758g'}

In [5]:
matches = pl.read_csv("../matches.csv")
match_tuples = [tuple(row) for row in matches.to_numpy()]
print(len(matches))
matches.head(10)

14995973


l_id,r_id
str,str
"""9#521172""","""8#487137"""
"""3#705514""","""7#705213"""
"""5#178036""","""7#746495"""
"""1#277832""","""3#956949"""
"""3#729809""","""9#516050"""
"""9#980653""","""5#980831"""
"""9#47980""","""0#807115"""
"""8#317525""","""3#317886"""
"""8#354120""","""0#353989"""


In [6]:
g = nx.Graph()
g.add_nodes_from(list(records.keys()))
g.add_edges_from(match_tuples)
print(len(g.nodes))
print(len(g.edges))

10000000
14995973


In [7]:
clusters = list(nx.connected_components(g))
print(len(clusters))
clusters[0]

6625848


{'0#0',
 '1#467487',
 '2#467081',
 '4#0',
 '5#0',
 '6#467762',
 '8#143205',
 '9#143417'}

In [8]:
import random

def mode_with_tiebreak(attr, vals, counter):
    counts = {v: counter[attr][v] for v in vals}
    max_count = max(counts.values())
    max_keys = [key for key, count in counts.items() if count == max_count]
    return random.choice(max_keys)

def produce_clean_record(records, cluster, counter):
    cluster_records = [records[r_id] for r_id in cluster]
    clean_record = dict()
    for a in list(cluster_records[0].keys()):
        if a == "_id":
            continue
        vals = [r[a] for r in cluster_records]
        val = mode_with_tiebreak(a, vals, counter)
        clean_record[a] = val
    return clean_record

In [9]:
import copy

clean_records = list()
i = 0
for c in clusters:
    if i % 100000 == 0:
        print(i)
    i += 1
    c = list(c)
    clean_record = produce_clean_record(records, c, value_counter)
    for r_id in c:
        record = copy.deepcopy(clean_record)
        record["_id"] = r_id
        clean_records.append(record)
print(len(clean_records))
print(clean_records[0])

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
10000000
{'given_name': 'annie', 'surname': 'johnson', 'suburb': 'warrenton', 'postcode': '27589', '_id': '2#467081'}


In [10]:
clean_ds = pl.DataFrame(clean_records).sort("_id")[["_id", "given_name", "surname", "suburb", "postcode"]]
print(len(clean_ds))
clean_ds.head(10)

10000000


_id,given_name,surname,suburb,postcode
str,str,str,str,str
"""0#0""","""annie""","""johnson""","""warrenton""","""27589"""
"""0#1""","""erin""","""pugh""","""asheboro""","""27205"""
"""0#10""","""robert""","""wilson""","""raleigh""","""27612"""
"""0#100""","""mary""","""stanley""","""carthage""","""28327"""
"""0#1000""","""marcus""","""mckoy""","""oxford""","""27565"""
"""0#10000""","""lee""","""dennis""","""albemarle""","""28001"""
"""0#100000""","""jennifer""","""jones""","""pinnacle""","""27043"""
"""0#100001""","""robert""","""dunn""","""kinston""","""28504"""
"""0#100002""","""miriam""","""richards""","""charlotte""","""28278"""


In [11]:
clean_ds = clean_ds.with_columns([
    pl.col("_id").str.split("#").list.get(0).cast(pl.Int64).alias("_id_1"),
    pl.col("_id").str.split("#").list.get(1).cast(pl.Int64).alias("_id_2")
])
clean_ds = clean_ds.sort(["_id_1", "_id_2"])
clean_ds = clean_ds.drop(["_id_1", "_id_2"])
clean_ds.head(10)

_id,given_name,surname,suburb,postcode
str,str,str,str,str
"""0#0""","""annie""","""johnson""","""warrenton""","""27589"""
"""0#1""","""erin""","""pugh""","""asheboro""","""27205"""
"""0#2""","""lauren""","""toledo""","""chapel hill""","""27514"""
"""0#3""","""georgetta""","""atkinson""","""goldsboro""","""27530"""
"""0#4""","""lynn""","""kitchens""","""greensboro""","""27408"""
"""0#5""","""jacqueline""","""lacewell""","""riegelwood""","""28456"""
"""0#6""","""mary""","""martin""","""charlotte""","""28215"""
"""0#7""","""gerald""","""clontz""","""midland""","""28107"""
"""0#8""","""marie""","""roach""","""reidsville""","""27320"""


In [12]:
clean_ds.write_csv("../dataset.csv")